# How do we prepare Yelp data to answer business questions?

In [1]:
import datetime
import json
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from shapely.geometry import Point, shape

## Goal (3 min)

In this case, we will introduce you to another uncleaned dataset for practice. We will focus on the nuances of cleaning the dataset parameter by parameter. Being able to dive into a single parameter and investigate its attributes will be crucial for modeling in later stages of the data science proceess.

## Introduction (5 min)

**Business Context.** [Yelp](https://www.yelp.com) is a very popular website, where anyone can write a review about restaurants, hotels, spas or any business. They have decided to start analysing the data and will use the results of the analysis to make decisions about new features to the service. They have now approached you, an independant data consultant with a requirement of cleaning the data they have and for you to help them get more context on the data.

**Business Problem.** Your task is to go through the data, and make transformations or additions to the data based on their needs, which are listed below.

**Analytical Context.** The client has shared three files with you containing details about the businesses and end-users on the service along with the reviews posted by these users. With this data, they would like you to do the following:

1. For each business, find out the county and state that it is a part of
2. For each business, calculate the total number of reviews received and the average star rating across those reviews
3. For each business, calculate the total number of check-ins by hour of day, by day of week, and across all time
    
We will initially run these operations locally with a small subset of the data. We will then run the same set of operations in an EC2 instance for the entire dataset.

## Reading in the data files (3 min)

Let's read in all three data files provided and familiarize ourselves with the data. Note that we truncate our read to the first 10000 rows due to the sizes of the files:

In [11]:
reviews = pd.read_csv('reviews.csv', nrows=1000)
businesses = pd.read_csv('businesses.csv', nrows=1000)
checkins = []
with open('checkins.json', encoding='utf-8')  as f:
    for row in f.readlines()[:1000]:
        checkins.append(json.loads(row))

The columns present in each of the files are:

1. **reviews**
    * **review_id** - Unique identifier for the review
    * **business_id** - Unique identifier of the business that is being reviewed
    * **user_id** - Unique identifier of the user who posted the review
    * **date** - The date and time of the review
    * **star** - Star rating for the review
    * **text** - Review text
    * **cool** - Number of cool votes received for the review
    * **funny** - Number of funny votes received for the review
    * **useful** - Number of funny votes received for the review
    
    
2. **businesses**
    * **business_id** - Unique identifier of the business
    * **name** - Name of the business
    * **categories** - Categories associated with the business
    * **latitude** - Location of the business (latitude)
    * **longitude** - Location of the business (longitude)
    * **review_count** - Number of reviews received for the business
    * **stars** - Average star rating, rounded to half stars
    
    
3. **checkins**
    * **business_id** - Unique identifier of the business
    * **date** - List of datetimes when users checked in to the business

## Handling null values in `reviews` (3 min)

As always, handling null values is a staple of cleaning datasets. Let's uncover the columns in the table `reviews` that contain null values:

In [12]:
reviews.isna().any()

business_id    False
cool            True
date            True
funny           True
review_id      False
stars           True
text           False
useful          True
user_id        False
dtype: bool

We can see that null values are present in the following columns: `date`, `stars`, `cool`, `funny`, and `useful`. Each of these columns need to be handled in a different way.

In contrast to the previous case, here we will look at some advanced methods for handling null values that don't use normal `pandas` operations. Let's get started.

## `date` (5 min)

Since the client has indicated they will likely be using the cleaned data for numerous analyses later on, removing rows with missing values or even writing in meaningless filler values will not work. So we need to figure out a sensible interpolation method.

### Exercise 1: (5 min)

Given what we know about the data so far, describe a sensible interpolation method that we can use.

**Answer.** From exploring the available data, we can surmise that reviews are ordered sequentially based on the `date` field. (Can you come up with a way to verify this via code?) Thus, we can fill in a reasonable value by interpolating between the immediate previous and immediate next records that are non-null.

However, our approach needs to handle cases where there are consecutive null values (we cannot always assume that the following row and the previous row are both not missing - sometimes there are long stretches of missing values!) In those cases, we will need to get the next and previous available dates which are non-null and perform the interpolation. A naive but reasonable assumption is that consecutive missing values are uniformly distributed within that interval, so we can do a simple linear interpolation based on the current row's relative position within its block of missing values. For example, if the current row is the 3rd row with a missing value in a block of 10 rows with missing values, then it should get an interpolated date which is $ \displaystyle\frac {3}{10 + 1} = \frac {3} {11} $ - ths of the way from the previous non-null date to the next non-null date.


Here is some code that implements our idea above:

In [13]:
nan_rows = reviews[reviews['date'].isnull()]
date_format = '%Y-%m-%d %H:%M:%S'

for index, row in nan_rows.iterrows():
    previous_date = reviews.iloc[index - 1]['date']
    if isinstance(previous_date, str):
        previous_date = datetime.datetime.strptime(previous_date, date_format)
    next_date = None
    next_date_count = 1
    while next_date is None:
        try:
            next_date = datetime.datetime.strptime(reviews.iloc[index + next_date_count]['date'], date_format)
        except Exception:
            next_date_count += 1
    difference = (next_date - previous_date).seconds
    current_date = previous_date + datetime.timedelta(seconds=difference/(next_date_count + 1))
    reviews.loc[index, 'date'] = current_date

_______

## `star` (10 min)

The next feature which has null values to handle is `star`. `star` is a numeric value and can range from 1 to 5, where 1 represents the lowest rating and 5 the highest. 

### Exercise 2: (10 min)

Describe an appropriate method for filling in the missing values of `star`.

**Answer.** Since we do not know how the client will want to use the values of `star` in later analysis, we do not want to use a naive method of filling in the values such as replacing nulls with the mean or median of the non-missing values. Doing so could compromise any analysis of the distribution of star ratings, as well as investigations of the relationships between star ratings and other variables.

One method that we discussed in the previous case, which is often a good choice, is to impute the missing values as a function of the other (non-missing) features of that row. For example, we could impute `star` given the values of `cool`, `date`, `funny`, `useful`, `text`. But this is not necessarily the wisest choice. For one thing, if those 5 variables are poor predictors of `star`, and in fact `star` is most closely related to other variables that we have not yet been provided, then we are pegging all future analyses involving `star` to a subpar interpolation moodel. And even if these variables are very good predictors of `star`, perhaps the client is most interested in examining outliers, which by definition won't be the values being filled in by the model.

Therefore, the most prudent option which will not alter subsequent summary statistics calculations and not skew the distribution of the non-missing data would be to replace all missing values with a standard `NaN`.

## `cool`, `funny`, `useful` and revisiting interpolation (10 min)

We can apply a similar logic to conclude that the `cool`, `funny`, and `useful` columns should also be filled with `NaN`s:

In [14]:
reviews['cool'].fillna(np.nan, inplace=True)
reviews['funny'].fillna(np.nan, inplace=True)
reviews['useful'].fillna(np.nan, inplace=True)

Now, let's assume, for the sake of progress, that the inputs `cool`, `date`, `funny`, `useful`, and `text` ARE good predictors for `star`, and that the client is mostly interested in high-level properties of the distributions of these variables and how they correlate to each other. In such a case, we can go ahead and leverage the power of machine learning to predict a reasonable estimate for `star`. 

We will do so by using the [IterativeImputer](https://scikit-learn.org/stable/modules/impute.html#multivariate-feature-imputation) class of `scikit-learn` to fill the missing values in these columns. `IterativeImputer` fills one column after another by building models for each column:

In [15]:
reviews.isnull().any()

business_id    False
cool            True
date           False
funny           True
review_id      False
stars           True
text           False
useful          True
user_id        False
dtype: bool

In [16]:
reviews['index'] = reviews.index
imputer = IterativeImputer()
imputed_df = pd.DataFrame(imputer.fit_transform(reviews[['index', 'stars']]))
imputed_df
imputed_df.columns = ['index', 'stars']
imputed_df.index = reviews.index
reviews[['stars']] = imputed_df['stars']
reviews.isnull().any()

business_id    False
cool            True
date           False
funny           True
review_id      False
stars          False
text           False
useful          True
user_id        False
index          False
dtype: bool

The value has to be dynamically filled, similar to the cool and funny columns.

### Dealing with erroneous values

Of course, missing values are not the only problem in datasets – erroneous values are too! So we should look a the other columns with non-null values to see if they could have errors that we can correct. Here, the other columns are `business_id`, `review_id`, `user_id`, and `text`. However, IDs are arbitrary identifiers which we have no way of ascertaining if they are correct or not, so we have to take the values we are given for granted.

`text` is the text of the user review, but the objective of any attempts to clean this is unclear - what metrics would we use to determine if a review text is "clean" or not? The answer to this question is not obvious at all without much more precise direction from the client about what they intend to use this text for, so we will leave it alone for now.

## Cleaning up `businesses` (15 min)

Let's start by checking which columns contain null values:

In [17]:
businesses.isnull().any()

business_id    False
categories      True
city           False
latitude       False
longitude      False
name           False
dtype: bool

We see that there are null values in `categories`.

### Exercise 3: (5 min)

Suppose for a moment that there were missing values in the `city` column. Can you describe a method which would allow us to effectively fill in missing `city` values?

**Answer.** Since we have the exact location coordinates (latitude and longitude) for each business, we can use an external dataset which contains the coordinates of the center of each city in the United States. We can then compute the distances between a particular business's coordinates and each city's coordinates and find the closest one, and fill in the missing value with that closest city's name.

### Exercise 4: (5 min)

Describe and implement the best method to fill in the null values in the `categories` column.

**Answer.** Since there are no other indicators in this file that could help determine the category of a restaurant, the null values in this column should be filled with a default replacement value like "Not Found", as shown below:

In [18]:
businesses['categories'].fillna('Not found', inplace=True)

It should be noted that this is not an ideal solution as those businesses do have a category associated with them; we just do not know what it is. It could be random that some businesses were not documented with a category but it is also possible that we are dealing with a systematic error here. For example, perhaps all the businesses in a specific area were not documented. Our default solution is the best option we have, but a good data scientist should note when his/her imputation methods are subject to the possibility of high errors and what additional information they would need to address those.

### Another look at erroneous values

Of course, missing values are not the only thing that can go wrong in a dataset – erroneous values can affect the analysis as well. Looking at the columns which we have not dealt with yet, we have `business_id`, `latitude`, `longitude`, and `name` as potential candidates.

### Exercise 5: (5 min)

Describe and implement suitable methods to deal with potential erroneous values in these columns.

**Answer.** `business_id` is just an arbitrarily assigned identifier, so we have no way of knowing if a non-missing value is wrong; we just have to take it for granted. The same goes with the `name` column. However, there are some basic rules we can apply to `latitude` and `longitude` to determine if they are off.

By basic interpretation of what latitude and longitude means, we know that the latitude values should be in the range `[-90, 90]` and longitude should be in the range `[-180, 180]`. So we should check that all data points have latitude and longitude values that lie within these ranges. When they are outside the range, we should replace them with acceptable nulls:

In [19]:
businesses.loc[(businesses['latitude'] < -90) | (businesses['latitude'] > 90), 'latitude'] = np.nan
businesses.loc[(businesses['longitude'] < -180) | (businesses['longitude'] > 180), 'longitude'] = np.nan

## Adding information for `businesses` (15 min)

On the client's request, we must find the county and state each business belongs in and add this to the table. We use the geoJSON files that contain the geoshapes of each state and county in the US. We will be using the [shapely](https://shapely.readthedocs.io/en/latest/manual.html) library to figure out whether a point is present inside the shape that represents the border of each state/county:

In [20]:
with open('us-state-shapes.json') as f:
    states = json.load(f)
    
def get_state_name(row):
    if not row['latitude'] or not row['longitude']:
        return None
    point = Point(row['longitude'], row['latitude'])
    for state in states['features']:
        polygon = shape(state['geometry'])
        if polygon.contains(point):
            return state['properties']['NAME']
        
businesses['state'] = businesses.apply(get_state_name, axis=1)

### Exercise 6: (5 min)

Find the county of each business and add them to the table.

**Answer.** One possible solution is given below:

In [21]:
with open('us-county-shapes.json') as f:
    counties = json.load(f)
    
def get_county_name(row):
    if not row['latitude'] or not row['longitude']:
        return None
    point = Point(row['longitude'], row['latitude'])
    for county in counties['features']:
        polygon = shape(county['geometry'])
        if polygon.contains(point):
            return county['properties']['NAME']
        
businesses['county'] = businesses.apply(get_county_name, axis=1)

We also need to add information about a restaurant's number of reviews and their average star rating.

### Exercise 7: (5 min)

Calculate the number of reviews received for each restaurant.

**Answer.** One possible solution is given below:

In [22]:
def calculate_review_count(row):
    business_id = row['business_id']
    business_reviews = reviews[reviews['business_id'] == business_id]
    return len(business_reviews)

businesses['review_count'] = businesses.apply(calculate_review_count, axis=1)

### Exercise 8: (5 min)

For each restaurant, calculate its avearge star rating.

**Answer.** One possible solution is given below:

In [23]:
def calculate_average_stars(row):
    business_id = row['business_id']
    business_reviews = reviews[reviews['business_id'] == business_id]
    average_stars = business_reviews['stars'].mean()
    return average_stars

businesses['stars'] = businesses.apply(calculate_average_stars, axis=1)

## Aggregating the check-ins data (20 min)

Now, let's do our final task: use the check-ins data given in the JSON file to compute various statistics on the number oof check-ins over various time periods.

### Exercise 9: (20 min)

Compute, for each business, the total number of check-ins by hour of day, by weekday, by day, and across all time.

**Answer.** One possible solution is given below:

In [24]:
checkin_aggregations = []
for checkin in checkins:
    business_info = {
        'id': checkin['business_id'],
        'total_checkins': len(checkin['date']),
        'checkins_by_day': {},
        'checkins_by_hour': {},
        'checkins_by_weekday': {},
    }
    for date in checkin['date'].split(','):
        date = datetime.datetime.strptime(date.strip(), date_format)
        day_str = date.strftime('%Y-%m-%d')
        checkin_for_day = business_info['checkins_by_day'].get(day_str, 0)
        checkin_for_day += 1
        business_info['checkins_by_day'][day_str] = checkin_for_day
        hour_checkins = business_info['checkins_by_hour'].get(date.hour, 0)
        hour_checkins += 1
        business_info['checkins_by_hour'][date.hour] = hour_checkins
        weekday_checkins = business_info['checkins_by_weekday'].get(date.weekday(), 0)
        weekday_checkins += 1
        business_info['checkins_by_weekday'][date.weekday()] = weekday_checkins
    checkin_aggregations.append(business_info)

## Copy notebook and data files to EC2 instance - Take Home Task (5 min)

We have performed these operations only on the first 10000 rows of our data tables. We will set up a Jupyter Notebook in an EC2 instance to apply these operations to the entire dataset. We need to open port 8889 on the EC2 instance.

Now, the process will not take as long this time, because we have already done most of the setup in the previous case. Let's copy the notebook file and the data files from the local machine to the EC2 instance. It can be done via an scp command, or using winSCP. There are different ways to SCP (secure copy) the file into the server:

1. If you are using a Linux or a Mac, type the following command to copy the file:

```
scp -i <path_to_pem_file> </path/to/jupyter/notebook> <username>@<EC2IP>:</destination/path>
```

2. If you are using Windows, you can either use WinSCP - https://winscp.net/eng/index.php or if you are using Putty, you can use the pscp command:

```
pscp -i <path_to_ppk_file> <\path\to\jupyter\notebook\> <username>@<EC2IP>:</destination/path>
```

## Execute the entire set with full data in the server - Take Home Task (5 min)

Once the file is copied, you can now access the jupyter notebook from `http:server_ip:8889`. Once you are able to access the file, just remove the `nrows=10000` parameter from the [read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) function call, in order to read the entire dataset. You can now click on `Cells > Run All` in order to run the same set of steps on the entire dataset.

## Conclusions (2 min)

In this case, we practiced our skills in dealing with missing values and also learned about common ways of handling erroneous values. We then creatively leveraged external data in order to engineer additional features based on client requests. We again took a small local sample of the overall data, then uploaded our work to Amazon EC2 so that they could be applied to the entire dataset.

## Takeaways (8 min)

In this case, you looked more at how to deal with missing data. You learned that although interpolation is a powerful tool, when and how to use it depends highly on the projected use cases of the data. Sometimes, it is better to simply replace a missing value with "Not found" or `NaN` so that it is clear to end users to discount it appropriately from subsequent analyses, rather than run the risk of skewing the resultant data distribution or summary statistics.

You also learned a few ways of using existing public data to help interpolate missing values in your current dataset. This is an invaluable skill and often times the best way to deal with missing values is to exercise this resourcefulness. It is important to recognize not all interpolation is made equal. If you have many parameters that are imputed poorly or with great uncertainty, it will be much harder to develop an accurate prediction model later if those parameters are key predictors.

Finally, you looked at erroneous values and various common ways these could leak into the data. You also learned that sometimes it is too difficult or impossible to systematically determine existence of error.
    
We dove into the technical nuances of data cleaning in this case but it is important to note this dataset is very simple compared to the data you will be working with in real life. It is not too uncommon to deal with datasets with hundreds of parameters where dozens have different types of missing/incorrect data. Additionally, you may encounter datasets where you are unsure what some columns/parameters mean. This dataset came with a nice handy dictionary explaining what information is in each column. 
    
Students are highly encouraged to thoroughly review all the EDA-related cases we have taught up to this point. Mastery of EDA will immensely improve the quality of your data wrangling & cleaning process, which in combination will impact your subsequent modeling process in a positive way.